In [2]:
import torch
from torch import nn
from transformers import Trainer, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from utils import LLAMA_TEMPLATES, MISTRAL_TEMPLATES, system_message
from dataset import RAGDataset
from tqdm import tqdm
import json
import yaml

/home/chris/Code/DeepLearning/AQGllama/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open('credentials.yaml', 'r') as file:
     credentials = yaml.safe_load(file)

In [4]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        inp = inputs[0]
        labels = inp.pop("labels")
        outputs = model(**inp)
        logits = outputs.get("logits")
        B, T, C = logits.shape
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(logits.view(B*T, C), labels.view(-1))
        return (loss, outputs) if return_outputs else loss


In [5]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

In [6]:
lora_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=32,
    bias="none",
    task_type="CAUSAL_LM",
)

In [7]:
base_model = {
    "llama": {
        "path": "meta-llama/Llama-2-7b-chat-hf",
        "save-path": "MediRAG-LLaMA",
        "templates": LLAMA_TEMPLATES
    },
    "mistral": {
        "path": "mistralai/Mistral-7B-Instruct-v0.2",
        "save-path": "MediRAG-Mistral",
        "templates": MISTRAL_TEMPLATES
    },
    "meditron": {
        "path": "epfl-llm/meditron-7b",
        "save-path": "MediRAG-Meditron",
        "templates": LLAMA_TEMPLATES
    }
}["llama"]

In [8]:
save_path = base_model['save-path']
message_templates = base_model['templates']
model = AutoModelForCausalLM.from_pretrained(
    base_model['path'], 
    quantization_config=quant_config, 
    use_auth_token=credentials['huggingface']
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

tokenizer  = AutoTokenizer.from_pretrained(
    base_model['path'], 
    use_auth_token=credentials['huggingface'],
    use_fast=False
)
tokenizer.pad_token = tokenizer.eos_token

/home/chris/Code/DeepLearning/AQGllama/.venv/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.06s/it]
/home/chris/Code/DeepLearning/AQGllama/.venv/lib/python3.11/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/chris/Code/DeepLearning/AQGllama/.venv/lib/python3.11/site-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [9]:
dataset = RAGDataset(
    tokenizer,
    system_message,
    LLAMA_TEMPLATES
)
print("Samples in dataset:", len(dataset))

Samples in dataset: 4609


In [10]:
training_params = TrainingArguments(
    output_dir=f"./{save_path}-Training",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=1,
    optim="paged_adamw_8bit",
    save_steps=50,
    logging_steps=1,
    learning_rate=2e-4,
    weight_decay=0.01,
    fp16=True,
    max_grad_norm=1.,
    warmup_ratio=0.03,
    lr_scheduler_type="linear"
)

In [11]:
trainer = CustomTrainer(
    model=model,
    args=training_params,
    train_dataset=dataset,
    data_collator=lambda x: x
)

In [12]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/chris/Code/DeepLearning/AQGllama/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.970300
2,0.789200
3,1.894000
4,0.959300
5,0.950900
6,0.622700
7,0.624700
8,1.048200
9,0.855800
10,0.778800


/home/chris/Code/DeepLearning/AQGllama/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/chris/Code/DeepLearning/AQGllama/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=145, training_loss=0.5561733878892043, metrics={'train_runtime': 621.4614, 'train_samples_per_second': 7.416, 'train_steps_per_second': 0.233, 'total_flos': 0.0, 'train_loss': 0.5561733878892043, 'epoch': 1.0})

In [13]:
trainer.save_model(f"{save_path}")